<a href="https://colab.research.google.com/github/sibyjackgrove/SolarPV-DER-simulation-utility/blob/master/examples/PV-DER_usage_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation and usage

## Installation from repository
Users can download the `SolarPV-DER-simulation-utility` repository and install with `pip`. Note that if the installation is done in a Google Colab environment, the kernel need to be restarted before importing the (otherwise you will get a module import error).

In [ ]:
import os
try:
    import pvder
except ImportError:
    !git clone https://github.com/sibyjackgrove/SolarPV-DER-simulation-utility.git
        
    if os.path.exists(os.path.join(os.getcwd(), 'SolarPV-DER-simulation-utility')):
        %cd SolarPV-DER-simulation-utility
        !pip install -e .
        %cd ..

## Basic usage

### Import classes
Import classes neccessary to define instances of the PV-DER model, events, simulation, and results.

In [1]:
from pvder.DER_wrapper import DERModel

from pvder.grid_components import Grid
from pvder.dynamic_simulation import DynamicSimulation
from pvder.simulation_events import SimulationEvents
from pvder.simulation_utilities import SimulationUtilities,SimulationResults
from pvder import utility_functions

#Import Python modules
import logging
import numpy as np
import scipy as sci
import math
import cmath
import pandas


import matplotlib as matplotlib
print('Scipy version:',sci.__version__)
print('Numpy version:',np.__version__)
print('Matplotlib version:',matplotlib.__version__)

%matplotlib inline    #To show interactive plot in the notebook cell 

Scipy version: 1.4.1
Numpy version: 1.20.3
Matplotlib version: 3.4.2


UsageError: unrecognized arguments: #To show interactive plot in the notebook cell


## Initialize voltage (for use with loop mode)

In [2]:
logging.debug('test')
Va=164.78-124.57j
Vb=-190.96-78.26j
Vc=26.24+206.56j
Vrms = abs(Va)/math.sqrt(2)
print('Vrms:{:.2f}'.format(Vrms))
print('Va:{:.2f},Vb:{:.2f},Vc:{:.2f}'.format(Va,Vb,Vc))
print('V0:{:.2f}'.format(Va+Vb+Vc))

events1 = SimulationEvents(verbosity = 'DEBUG')
grid1 = Grid(events=events1,unbalance_ratio_b=1.0,unbalance_ratio_c=1.0)


Vrms:146.07
Va:164.78-124.57j,Vb:-190.96-78.26j,Vc:26.24+206.56j
V0:0.06+3.73j


## Select options

In [3]:
STAND_ALONE = True
STEADY_STATE = True
LOOP=False

model_type = 'ThreePhaseUnbalanced' #Model type
#three phase model types: 'ThreePhaseBalanced','ThreePhaseUnbalanced','ThreePhaseUnbalancedConstantVdc'
#single phase model types: 'SinglePhase'

der_verbosity = 'DEBUG' #Logging verbosity
config_file = r'../config_der.json'
parameter1 = '50'#'50_type1'#'50','10' '50_constantVdc','50_balanced','250'

### Create objects required to simulate either single phase or three phase PV-DER

In [4]:
if STAND_ALONE:
    PV_DER1 = DERModel(modelType=model_type,events=events1,configFile=config_file,
                       gridModel=grid1,
                       derId=parameter1,
                       standAlone = STAND_ALONE,steadyStateInitialization=STEADY_STATE)   
else:
    PV_DER1 = DERModel(modelType=model_type,events=events1,configFile=config_file,
                       Vrmsrated = Vrms,
                       gridVoltagePhaseA = Va,gridVoltagePhaseB = Vb,gridVoltagePhaseC = Vc,gridFrequency=2*math.pi*60.0,
                       derId=parameter1,
                       standAlone = STAND_ALONE,steadyStateInitialization=STEADY_STATE,
                       verbosity = der_verbosity)        
    
sim1 = DynamicSimulation(gridModel=grid1,PV_model=PV_DER1.DER_model,events = events1,verbosity = 'INFO',solverType='odeint',LOOP_MODE=LOOP) #'odeint','ode-vode-bdf'
results1 = SimulationResults(simulation = sim1,PER_UNIT=True,verbosity = 'INFO')
PV_DER1.DER_model.show_PV_DER_parameters('controller_gains')

INFO:pvder_logger:DER configuration with ID:50 was found in ../config_der.json
INFO:pvder_logger:Updated DER argument verbosity from DER_config
INFO:pvder_logger:-SolarPVDERThreePhase_1:Instance created with DER parameter ID: 50; Specifications - Srated:50.0 kVA, Ppv:45.8 kW, Vrms:177.0 V, Steady state:True,LVRT Enable:True,HVRT Enable:True


Optimization results:[0.91125869 0.02939627 1.21614206 0.03366481]
----Parameters for DER with ID:50----
Current controller :
Kp_GCC:6000.000
Ki_GCC:2000.000
wp:200000.000
DC link voltage controller :
Kp_DC:-2.000
Ki_DC:-10.000
Reactive power controller :
Kp_Q:0.200
Ki_Q:10.000


### Show ridethrough settings

In [5]:
PV_DER1.DER_model.show_RT_settings(settings_type='LVRT',PER_UNIT=True)


______-SolarPVDERThreePhase_1 - LVRT_____
______Flags______
LVRT_ENABLE:True
LVRT_TRIP:False,LVRT_MOMENTARY_CESSATION:False 
______Thresholds______
Vrms_ref:177.00 V
Zone:1,Vthreshold:0.88,tthreshold:5.00,tminridethrough:20.000,mode:momentary_cessation
Zone:2,Vthreshold:0.70,tthreshold:3.50,tminridethrough:2.500,mode:momentary_cessation
Zone:3,Vthreshold:0.50,tthreshold:0.25,tminridethrough:1.000,mode:momentary_cessation
OUTPUT_CESSATION_DELAY:0.01,OUTPUT_RESTORE_DELAY:0.1


### Updating voltage ridethrough (VRT) settings
1. VRT settings are stored in pvderConfig dictionary
2. Update the dictionary by providing appropriate key and value
3. Call VRT_initialize() method.
4. Call show_RT_settings() to make sure that settings have changed

In [6]:
#print(PV_DER1.pvderConfig.keys()) #Print contents of dictionary
#PV_DER1.pvderConfig.update({'t_LV1_limit': 1.5})
#PV_DER1.VRT_initialize()
#PV_DER1.show_RT_settings(settings_type='LVRT',PER_UNIT=True)

### Add or remove events during simulation

In [7]:
#Voltage sag event
events1.add_grid_event(6.0,Vgrid=0.68,Vgrid_angle =0.0,fgrid=60.0)
#Voltage rise event
events1.add_grid_event(9.0,Vgrid=1.0,Vgrid_angle =0.0,fgrid=60.0)

### Show events

In [8]:
events1.show_events()

Showing all event in events instance -events_1
Total solar events:0
Total grid events:2
t:6.000, Grid event, Grid voltage: Magnitude:0.68, Angle:0.000 V, Frequency is 60.00
t:9.000, Grid event, Grid voltage: Magnitude:1.00, Angle:0.000 V, Frequency is 60.00


### Specify run time flags, simulation time, and run simulation 

In [9]:
#%%time
PV_DER1.MPPT_ENABLE=False
PV_DER1.RAMP_ENABLE = False
PV_DER1.VOLT_VAR_ENABLE = False
PV_DER1.LVRT_ENABLE = True
PV_DER1.HVRT_ENABLE = True
PV_DER1.LFRT_ENABLE = False
PV_DER1.DO_EXTRA_CALCULATIONS = True
sim1.jacFlag = True
sim1.DEBUG_SIMULATION = False
sim1.DEBUG_VOLTAGES = True
sim1.DEBUG_CURRENTS = True
sim1.DEBUG_POWER = False
sim1.DEBUG_CONTROLLERS  = True
sim1.DEBUG_PLL = False
sim1.PER_UNIT = True
sim1.DEBUG_SOLVER  = True
sim1.tStop = 20.0
sim1.tInc = 0.001
sim1.run_simulation()
PV_DER1.DER_model.validate_model()
PV_DER1.DER_model.show_PV_DER_states('voltage')
PV_DER1.DER_model.show_PV_DER_states('current')
PV_DER1.DER_model.show_PV_DER_states('power')
PV_DER1.DER_model.show_PV_DER_states('duty cycle')

None-sim_1:Simulation started at 0.0 s and will end at 20.0 s


INFO:pvder_logger:-SolarPVDERThreePhase_1:6.0011:LV1 zone entered at 6.0011 s for 120.508 V p.u. (Vref:177.00 V,V_thresh:155.76 V(0.88),t_thresh:5.00 s,t_min_ridethrough:20.00 s,mode:momentary_cessation)
INFO:pvder_logger:-SolarPVDERThreePhase_1:6.0011:LV2 zone entered at 6.0011 s for 120.508 V p.u. (Vref:177.00 V,V_thresh:123.90 V(0.70),t_thresh:3.50 s,t_min_ridethrough:2.50 s,mode:momentary_cessation)
INFO:pvder_logger:-SolarPVDERThreePhase_1:8.5031:LV2 zone - momentary cessation after 2.5021 s for 120.476 V p.u. (Vref:177.00 V,V_thresh:123.90 V(0.70),t_thresh:3.50 s,t_min_ridethrough:2.50 s,mode:momentary_cessation)
INFO:pvder_logger:-SolarPVDERThreePhase_1:8.5031:DER disconnect timer started.
INFO:pvder_logger:-SolarPVDERThreePhase_1:8.5198:DER will be disconnected (t_disconnect_timer:0.0167 s,Vrms measured = 120.4762 V,LVRT Momentary cessation:True,LVRT Trip:False,HVRT Momentary cessation:False,HVRT Trip:False)
INFO:pvder_logger:-SolarPVDERThreePhase_1:9.0007:LV1 flag reset at 9.0

-SolarPVDERThreePhase_1:Time:[0.0000e+00 1.0000e-03 2.0000e-03 ... 1.9998e+01 1.9999e+01 2.0000e+01],Methods:[2 2 2 ... 2 2 2],Time steps:[0.00044835 0.00150291 0.00150291 ... 0.00833333 0.00833333 0.00833333]
-SolarPVDERThreePhase_1:Simulation successful for all time steps!
None-sim_1:Simulation was completed in 00:00:11
Active power output error:0.5873
Reactive power output error:0.0026
Inverter filter active power loss error:0.0000
Inverter filter reactive power loss error:0.0000

______-SolarPVDERThreePhase_1 - Voltage_____
Vdc:583.03
Vta:250.59+7.78j V
Vtb:-118.56-220.91j V,Vtb:-132.04+213.13j V
Vtn:0.00+0.00j V
Va:250.39+6.67j V
Vb:-119.42-220.18j V,Vc:-130.97+213.51j V
Vn:0.00+0.00j V
Vtrms:177.28 V
Vpccrms:177.11 V

______-SolarPVDERThreePhase_1 - Current_____
ia:117.08+3.12j A
ib:-55.84-102.95j A,ic:-61.24+99.83j A
In:0.00-0.00j A
Irms:82.82 V

______-SolarPVDERThreePhase_1 - Power_____
Ppv:44045.2 W
S:44045.2+193.9j VA
S_PCC:44004.1+0.0j VA

______-SolarPVDERThreePhase_1 - Du

### Sequence of events from plot
1. Low voltage event causes grid voltage to drop below 0.7 p.u. at 6.0 s.
2. DER rides through fault from 6.0 s to 7.0 s.
3. DER enters momentary cessation at 7.0 s.
3. Voltage recovers back to 1 p.u. at 9.0 s.
4. DER output recovers to rated power output at 9.5 s.

In [ ]:
results1.PER_UNIT = True
results1.PLOT_TITLE = True
results1.font_size = 18
results1.plot_DER_simulation(plot_type='active_power_Ppv_Pac_PCC')#

In [ ]:
results1.plot_DER_simulation(plot_type='reactive_power')#_Q_PCC

In [ ]:
results1.plot_DER_simulation(plot_type='voltage_LV') #voltage_Vpcclv

In [ ]:
results1.plot_DER_simulation(plot_type='voltage_Vdc')

In [ ]:
results1.plot_DER_simulation(plot_type='current')

In [ ]:
results1.plot_DER_simulation(plot_type='duty_cycle')